In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup as BS
import re

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/abigailasper/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_49215/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## log in to GoodReads

In [3]:
driver.get("https://www.goodreads.com/user/sign_in")

In [4]:
#select "sign in with email" option
#https://www.geeksforgeeks.org/click-button-by-text-using-python-and-selenium/
button = driver.find_element_by_link_text("Sign in with email")
button.click()

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_49215/3637183351.py:1: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  button = driver.find_element_by_link_text("Sign in with email")


In [7]:
username = driver.find_element(By.NAME, "email")
username.send_keys("abigailasper@gmail.com")

### import json & keys file to safely import GoodReads password

In [9]:
import json

In [12]:
with open('../keys/gr_pw.json') as fi:
    credentials = json.load(fi)

In [13]:
gr_pw = credentials['gr_pw']

In [14]:
password = driver.find_element(By.NAME, "password")
password.send_keys(gr_pw)

In [15]:
#submit login credentials
signin = driver.find_element(By.ID, "signInSubmit")
signin.click()

## navigate to desired shelf (Booktok Books) & begin scrape

In [16]:
driver.get("https://www.goodreads.com/shelf/show/booktok")

In [26]:
#testing
book_list = []

for page in range(1, 3, 1):
    
    page_url = "https://www.goodreads.com/shelf/show/booktok?page=" + str(page)
    driver.get(page_url)
    title = driver.find_elements(By.CLASS_NAME, "bookTitle")
    author = driver.find_elements(By.CLASS_NAME, "authorName")
  
    for i in range(len(title)):
        book_list.append([title[i].text, author[i].text])
  
print(book_list)
        

[['The Seven Husbands of Evelyn Hugo (Hardcover)', 'Taylor Jenkins Reid'], ['It Ends with Us (Kindle Edition)', 'Colleen Hoover'], ['The Song of Achilles (Paperback)', 'Madeline Miller'], ['The Invisible Life of Addie LaRue (Hardcover)', 'V.E. Schwab'], ['The Cruel Prince (The Folk of the Air, #1)', 'Holly Black'], ['From Blood and Ash (Blood and Ash, #1)', 'Jennifer L. Armentrout'], ['Six of Crows (Six of Crows, #1)', 'Leigh Bardugo'], ['They Both Die at the End (Hardcover)', 'Adam Silvera'], ['A Court of Thorns and Roses (A Court of Thorns and Roses, #1)', 'Sarah J. Maas'], ['Red, White & Royal Blue (Paperback)', 'Casey McQuiston'], ['The Love Hypothesis (Paperback)', 'Ali Hazelwood'], ['Ugly Love (Kindle Edition)', 'Colleen Hoover'], ['We Were Liars (Kindle Edition)', 'E. Lockhart'], ['People We Meet on Vacation (Paperback)', 'Emily Henry'], ['The Spanish Love Deception (ebook)', 'Elena Armas'], ['The Hating Game (ebook)', 'Sally Thorne'], ['Beach Read (Paperback)', 'Emily Henry'], 

In [24]:
print(titles)

<selenium.webdriver.remote.webelement.WebElement (session="09d78b270073467952613301e14d6c3a", element="22e52f86-2fcb-40e4-b0d6-0f4428cf1a30")>
